In [1]:
import os
# dirname_number = 0
# file_number = 1
# writefile = open('NewGroup_data.txt', 'w')
# for dirname, subdir, filename in os.walk('C:/Users/karis/Desktop/latently/20news-bydate-train/', topdown=True):
#     if dirname != "C:/Users/karis/Desktop/latently/20news-bydate-train/":
#         for file_nam in filename:
#             f = os.path.join(dirname, file_nam)
#             with open(f,'r') as main:
#                 contents = main.readlines()
#                 writefile.write("{}\t{}\t{}\n".format(file_number, contents, dirname_number))
#                 file_number = file_number+1
#         dirname_number = dirname_number+1
# writefile.close()


In [11]:
group_names = []
for dirname, subdir, filename in os.walk('C://Users//karis//Desktop//latently//20news-bydate-train', topdown=False):
     for i in  subdir:
            group_names.append(i)


In [3]:
import pandas as pd
import numpy as np
import os
from pandas import Series, DataFrame
group_train = pd.read_csv("C:\\Users\\karis\\NewGroup_data.txt", sep="\t", header = None, names= ['phrase_id', 'review', 'group_id'])
group_train.head()

,phrase_id,review,group_id
0,1,"['From: mathew <mathew@mantis.co.uk>\n', 'Subj...",0
1,2,"['From: mathew <mathew@mantis.co.uk>\n', 'Subj...",0
2,3,['From: I3150101@dbstu1.rz.tu-bs.de (Benedikt ...,0
3,4,"['From: mathew <mathew@mantis.co.uk>\n', 'Subj...",0
4,5,"['From: strom@Watson.Ibm.Com (Rob Strom)\n', '...",0


In [4]:
group_test = pd.read_csv("C:\\Users\\karis\\NewGroup_data_test.txt", sep="\t", header = None, names= ['phrase_id', 'review', 'group_id'])
group_test.head()

,phrase_id,review,group_id
0,1,['From: decay@cbnewsj.cb.att.com (dean.kaflowi...,0
1,2,"['From: cfaehl@vesta.unm.edu (Chris Faehl)\n',...",0
2,3,"['From: mathew <mathew@mantis.co.uk>\n', 'Subj...",0
3,4,"['From: dps@nasa.kodak.com (Dan Schaertel,,,)\...",0
4,5,"['From: halat@panther.bears (Jim Halat)\n', 'S...",0


In [5]:
# maping cateogories to train dataframe

group_train['category'] = group_train.group_id.map({0:'alt', 1:'comp',2:'comp', 3:'comp', 4:'comp',5:'comp',6:'misc', 7:'rec', 8:'rec',9:'rec', 10:'rec', 11:'sci',
                                    12:'sci', 13:'sci', 14:'sci', 15:'religion', 
                                    16:'politics', 17:'politics', 18:'politics', 19:'religion'})

# maping cateogories to test dataframe

group_test['category'] = group_test.group_id.map({0:'alt', 1:'comp',2:'comp', 3:'comp', 4:'comp',5:'comp',6:'misc', 7:'rec', 8:'rec',9:'rec', 10:'rec', 11:'sci',
                                    12:'sci', 13:'sci', 14:'sci', 15:'religion', 
                                    16:'politics', 17:'politics', 18:'politics', 19:'religion'})

group_test.shape

(7532, 4)

In [12]:
# selecting categories 
group_train_category = group_train[(group_train.category == 'comp') | (group_train.category == 'rec') | (group_train.category == 'politics') | (group_train.category == 'religion')]
group_test_category = group_test[(group_test.category == 'comp') | (group_test.category == 'rec') | (group_test.category == 'politics') | (group_test.category == 'religion')]


# spliting data into input X_train and output Y_train
X_train = group_train_category.review
# Y_train = group_train_category.category

# spliting data into input X_train and output Y_train
X_test = group_test_category.review
# Y_test = group_test_category.category

print(X_train.shape)
print(X_test.shape)

(7876,)
(5244,)


In [8]:
from sklearn.preprocessing import LabelEncoder

Label_encoded = LabelEncoder()

Y_train_label_encoded = Label_encoded.fit_transform(group_train_category['category'].values)
Y_test_label_encoded = Label_encoded.fit_transform(group_test_category['category'].values)


print(Y_train_label_encoded.shape)
print(Y_test_label_encoded.shape)



(7876,)
(5244,)


In [13]:
def tokenize_test_20New(vect):
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import accuracy_score
    

    # fit and transform X_train

    X_train_dtm = vect.fit_transform(X_train)
    X_test_dtm = vect.transform(X_test)

    LR = LogisticRegression(multi_class= 'ovr')

    LR.fit(X_train_dtm, Y_train_label_encoded)

    Y_predicted = LR.predict(X_test_dtm)

    print ('accracy:', accuracy_score(Y_test_label_encoded, Y_predicted))



In [14]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
tokenize_test_20New(vect)

accracy: 0.944889397407


In [20]:
# include 1-grams and 2-grams
vect = CountVectorizer(ngram_range=(1, 2))
tokenize_test_20New(vect)

accracy: 0.941266209001


In [173]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
tokenize_test_20New(vect)

('accracy:', 0.93821510297482835)


In [21]:
def tokenize_test_20New_svm(vect):
    from sklearn import svm
    from sklearn.metrics import accuracy_score
    

    # fit and transform X_train

    X_train_dtm = vect.fit_transform(X_train)
    X_test_dtm = vect.transform(X_test)

    lin_clf = svm.LinearSVC()
    lin_clf.fit(X_train_dtm, Y_train_label_encoded)
    Y_predicted = lin_clf.predict(X_test_dtm)

    print ('accracy:', accuracy_score(Y_test_label_encoded, Y_predicted))

In [22]:
# BAG OF WORDS SUPPORT VECTOR MACHINE CLASSIFICATION

from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
tokenize_test_20New_svm(vect)

accracy: 0.936880244088


In [23]:
# include 1-grams and 2-grams
vect = CountVectorizer(ngram_range=(1, 2))
tokenize_test_20New_svm(vect)

accracy: 0.941075514874
